In [125]:
import os
import numpy as np
import pandas as pd
import random as rand

base_path = "instances"

##period req_period compu_time server cpu hd bd
user_path = "users"
user_file = 'users-melbcbd-generated.csv'
user = pd.read_csv(os.sep.join((base_path,user_path,user_file)))

nbperiods = 100

user['Arrival_Time'] = np.random.randint(1, nbperiods, user.shape[0])
user['Computation_Time'] = np.random.randint(1, nbperiods*0.2, user.shape[0])
user['Due_Time'] = user['Arrival_Time'] + user['Computation_Time'] + np.random.randint(1, nbperiods*0.1, user.shape[0])
display(user)

,Latitude,Longitude,Arrival_Time,Computation_Time,Due_Time
0,-37.814619,144.974443,16,8,32
1,-37.810140,144.970454,98,8,111
2,-37.819892,144.957305,41,4,48
3,-37.814524,144.953632,5,11,25
4,-37.814100,144.963000,85,16,106
...,...,...,...,...,...
811,-37.813049,144.957795,63,11,82
812,-37.815577,144.963441,67,7,78
813,-37.808372,144.969200,87,3,97
814,-37.809588,144.968573,82,9,95


In [126]:
resources = 10000

server_path = "edge-servers"
server_file = "site-optus-melbCBD.csv"
edge_servers = pd.read_csv(os.sep.join((base_path,server_path,server_file)))
edge_servers = edge_servers[['LATITUDE','LONGITUDE','SITE_PRECISION']]
edge_servers['SITE_PRECISION'] = edge_servers['SITE_PRECISION'].str.replace('Within 10 meters','10')
edge_servers['CPU'] = np.random.randint(5000, resources, edge_servers.shape[0])
edge_servers['HDD'] = np.random.randint(5000, resources, edge_servers.shape[0])
edge_servers['BDW'] = np.random.randint(5000, resources, edge_servers.shape[0])
edge_servers['TYPE'] =  np.random.rand(edge_servers.shape[0])

threshold = 0.7
edge_servers['TYPE'] = edge_servers['TYPE'].map(lambda x:1 if x>=threshold else x)
display(edge_servers)

#print(edge_servers[edge_servers['TYPE'] == 1.0].count())


user['CPU'] =  np.random.randint(1, resources*0.01, user.shape[0])
user['HDD'] =  np.random.randint(1, resources*0.01, user.shape[0])
user['BDW'] =  np.random.randint(1, resources*0.01, user.shape[0])

,LATITUDE,LONGITUDE,SITE_PRECISION,CPU,HDD,BDW,TYPE
0,-37.815170,144.974760,10,5381,6136,7799,0.328558
1,-37.815240,144.952560,10,7372,6090,5247,1.000000
2,-37.812390,144.971200,10,8630,6415,8465,1.000000
3,-37.816790,144.969180,10,8205,6280,6238,0.595612
4,-37.818080,144.956920,10,8129,7677,9566,1.000000
...,...,...,...,...,...,...,...
120,-37.816496,144.965090,10,7424,6987,6717,0.025505
121,-37.815296,144.958369,10,6500,6942,7854,0.243196
122,-37.816310,144.964351,10,7635,6735,9003,0.470999
123,-37.817386,144.961230,10,7133,9928,9744,0.433324


In [127]:
n = 10
m = 5
users_n = user.sample(10).reset_index()
servers_n = edge_servers.sample(5).reset_index()

In [128]:
import math

radio = 1000

def distance(server,user=user):
    
#     Convert the latitudes
#     and longitudes
#     from degree to radians.
    lat1 = math.radians(user['Latitude'])
    long1 = math.radians(user['Longitude'])
    lat2 = math.radians(server['LATITUDE'])
    long2 = math.radians(server['LONGITUDE'])
     
    #// Haversine Formula
    dlong = long2 - long1
    dlat = lat2 - lat1
    ans = math.pow(math.sin(dlat / 2), 2) + math.cos(lat1) * math.cos(lat2) * math.pow(math.sin(dlong / 2), 2)
    ans = 2 * math.asin(math.sqrt(ans))
    R = 6371     
    #     // Calculate the result
    ans = ans * R 
    return ans*1.609344*1000 # convert from milles to meters

def distance_servers(user):
    s=""
    for indx,server in servers_n.iterrows():
        if distance(server,user)<= radio:
            s+=str(indx)+"_"
    if s=="":
        s=str(random.randint(user_n.shape[0]))
    return s[:-1]

users_n["Servers"] = users_n.apply(distance_servers,axis=1)
# display(users_n)
# display(servers_n)

In [133]:
users_n = users_n[['Arrival_Time','Computation_Time','Due_Time','CPU','HDD','BDW','Servers']]
servers_n = servers_n[['CPU','HDD','BDW','TYPE']]
with open(f'fog_computing_{nbperiods}_{m}_{n}_input.txt','w') as file:
    head = f"# periods servers users\n{nbperiods}\t{m}\t{n}\n"
    file.write(head)
    file.write('''# servers
# cpu hd bd type_server:
# type server: dedicated = 1, volunteer good = 0.9-0.7, neither 0.6-0.4, bad = 0.3-0.1 ---> % of nb of periods
# p.e 100*0.1 ---> E[x] = 1/10 ---> interval of time of 10 between events (swith on/off)''')
    server_str = ""
    for indx,server in servers_n.iterrows():
        for val in server:
            server_str+=str(val)+"\t"
        server_str=server_str[:-1]
        server_str+="\n"
    file.write(server_str[:-1])
    file.write("\n")
    file.write('''#users
#arrival compu_time req_period cpu hd bd servers\n''')
    user_str = ""
    for indx,user in users_n.iterrows():
        for val in user:
            user_str += str(val)+"\t"
        user_str = user_str[:-1]
        user_str += "\n"
    file.write(user_str[:-1])

In [130]:
display(servers_n)
users_n

,CPU,HDD,BDW,TYPE
0,8630,6415,8465,1.000000
1,7048,9031,6239,1.000000
2,9315,9679,7950,0.127791
3,9602,6248,5259,0.596330
4,6096,7374,6538,0.490561


,Arrival_Time,Computation_Time,Due_Time,CPU,HDD,BDW,Servers
0,68,6,75,21,49,4,2_3_4
1,89,1,99,41,79,96,0
2,57,1,67,69,29,98,2_3_4
3,44,16,62,51,56,39,0_1
4,74,4,84,49,97,65,1
5,84,19,109,41,98,22,1_2
6,31,6,42,75,9,39,2_3_4
7,5,17,29,8,55,54,0_1
8,60,18,86,55,45,11,0_1
9,75,5,82,79,77,63,0_1


In [166]:
import random as rand
nbperiodos = 100
p=0.1       #mean => 90

lambda_param = 1/(p*nbperiodos)

s = 0
it = 1000
for i in range(it):
    s += rand.expovariate(lambda_param)
print(s/it)

10.278523303410443
